In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.amp as amp

from tqdm import tqdm

from torch.utils.data import DataLoader, Dataset
from torch.amp import autocast, GradScaler
from torch.optim import AdamW

from transformers import BartModel
from transformers import BartModel, BartTokenizer
from transformers import BartConfig, BartModel
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer, AutoModel

from tokenizers import Tokenizer

In [2]:
import torch
import os

num_workers = os.cpu_count() // 2  # CPU 개수의 절반 사용 (보통 최적)
print(f"추천 num_workers 값: {num_workers}")

# 현재 토크나이저 병열화와, num_workers를 사용하는 병렬화가 충동일 일으킴으로, 토크나이저 병렬화 기능을 제거하고시돟.
os.environ["TOKENIZERS_PARALLELISM"] = "false"



추천 num_workers 값: 6


## 사전학습된 koBart, train_encoder 가져오기.

샘플 텍스트를 각각 1줄씩 넣어보기.

## 데이터셋 정의하기

In [3]:
# 데이터셋 정의
class DecoderDataset(Dataset):
    def __init__(self, df_path):
        df = pd.read_csv(df_path)
        self.inputs = df["input"].tolist()
        self.outputs = df["output"].tolist()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = self.inputs[idx]
        output_text = self.outputs[idx]
        return input_text, output_text

In [4]:
# kobart, kobart_tokenizer 가져오는 코드
def get_kobart_and_tokenizer():
    kobart_tokenizer = AutoTokenizer.from_pretrained("gogamza/kobart-base-v2")
    kobart_model = AutoModelForSeq2SeqLM.from_pretrained("gogamza/kobart-base-v2")
    return kobart_tokenizer, kobart_model


# mybart, mybart_tokenizer 가져오는 코드
def get_mybart_and_tokenizer(tokenizer_path, model_path, model_config):
    mybart_tokenizer = Tokenizer.from_file(tokenizer_path)
    # BART 모델 설정
    mybart = BartModel(model_config)  # BART 모델 생성

    # 저장된 textEncoder 모델 가중치 불러오기
    state_dict = torch.load(model_path, map_location="cpu")
    # BART 모델의 인코더 부분에만 가중치 로드
    mybart.encoder.load_state_dict(state_dict, strict=False)

    print("✅ BART 인코더 가중치 로드 완료!")
    return mybart_tokenizer, mybart




In [5]:
# input1(난독화 텍스트) 임베딩 벡터 생성용 함수 (배치 단위 지원)
def get_encodedKr_emb_vec(input_texts, mybart_tokenizer, mybart_model, device, max_length=1026):
    '''
    mybart_model는 eval() 모드여야 한다.
    input_texts: List of input strings (배치 단위)
    '''
    input_ids_batch = []
    attention_mask_batch = []

    for input_text in input_texts:
        input_encoded = mybart_tokenizer.encode(input_text)
        input_ids = input_encoded.ids[:max_length]
        attention_mask = [1] * len(input_ids)

        pad_id = mybart_tokenizer.token_to_id("<pad>")
        input_ids += [pad_id] * (max_length - len(input_ids))
        attention_mask += [0] * (max_length - len(attention_mask))

        input_ids_batch.append(input_ids)
        attention_mask_batch.append(attention_mask)

    # (batch_size, seq_len) 형태로 변환
    input_ids = torch.tensor(input_ids_batch, dtype=torch.long).to(device)
    attention_mask = torch.tensor(attention_mask_batch, dtype=torch.long).to(device)

    input_emb = mybart_model.encoder(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
    return input_emb  # (batch_size, seq_len, hidden_dim)

# input2(한국어 텍스트) 임베딩 벡터 생성용 함수 (배치 단위 지원)
def get_korean_emb_vec(output_texts, kobart_tokenizer, kobart_model, device, max_length=1026):
    '''
    kobart_model는 eval() 모드여야 한다.
    output_texts: List of output strings (배치 단위)
    '''
    output_ids_batch = kobart_tokenizer(output_texts, return_tensors="pt", padding="max_length",
                                        max_length=max_length, truncation=True)["input_ids"].to(device)

    output_emb = kobart_model(output_ids_batch).encoder_last_hidden_state
    return output_emb  # (batch_size, seq_len, hidden_dim)


## 디코더의 네트워크 구조 만들기

In [6]:
class CrossAttentionDecoder(nn.Module):
    def __init__(self, 
                 kobart_tokenizer_vocab_size, 
                 hidden_dim=768, 
                 num_layers=6, 
                 num_heads=8, 
                 dropout=0.1):
        
        super(CrossAttentionDecoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.kobart_tokenizer_vocab_size = kobart_tokenizer_vocab_size

        # Transformer Decoder Layer에 dropout 추가
        decoder_layer = nn.TransformerDecoderLayer(d_model=hidden_dim, nhead=num_heads, dropout=dropout)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)

        # Fully Connected Layers
        self.fc1 = nn.Linear(hidden_dim, hidden_dim * 4)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)  # Dropout 추가
        self.fc_out = nn.Linear(hidden_dim * 4, kobart_tokenizer_vocab_size)
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, input1, input2, tgt_mask=None):
        """
        input1: 난독화된 텍스트 임베딩 벡터 (Key, Value) -> (batch_size, seq_len, hidden_dim)
        input2: 복원된 한국어 텍스트 임베딩 벡터 (Query) -> (batch_size, seq_len, hidden_dim)
        tgt_mask: (batch_size, seq_len, seq_len) 크기의 마스크 텐서 (외부에서 제공)
        """
        decoder_output = self.decoder(input2, input1, tgt_mask=tgt_mask)

        x = self.fc1(decoder_output)
        x = self.relu(x)
        x = self.dropout(x)  # Dropout 적용
        x = self.fc_out(x)

        return self.softmax(x)


## 훈련 에포크 작성하기

#### 일단 1개의 배치만 현재 네트워크에 넣어보기

In [7]:
# 데이터셋, 데이터로더 정의
datset_path = 'datas/decoder_augmentation.csv'
decoder_dataset = DecoderDataset(df_path= datset_path)
dataloader = DataLoader(
    decoder_dataset, 
    batch_size=2, 
    shuffle=True, 
    num_workers=4, 
    pin_memory=True,
)

dataiter = iter(dataloader)


In [8]:
# 사전학습된 토크나이저, BART인코더 가져오기

# 저장해놓은 모델의 네트워크 구조와 동일해야한다.
model_config = BartConfig(
    vocab_size=50000,
    d_model=768,
    encoder_layers=4,
    encoder_attention_heads=8,
    max_position_embeddings=1026,
)
tokenizer_path = "tokenizers/BPE_tokenizer_50000_aug.json"
model_path = "trained_encoder3.pth"

# 토크나이저, 모델을 불러오는 코드.
kobart_tokenizer, kobart_model = get_kobart_and_tokenizer()
mybart_tokenizer, mybart_model = get_mybart_and_tokenizer(tokenizer_path, model_path, model_config)

# kobart_tokenizer.max_len_single_sentence, kobart_tokenizer.vocab_size # (1000000000000000019884624838656, 30000)
# kobart_model.config.d_model, kobart_model.config.max_position_embeddings # (768, 1026)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
/tmp/ipykernel_150021/1922565671.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbit

✅ BART 인코더 가중치 로드 완료!


In [9]:
# 모델들의 모드 설정 및 device 설정.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 사전학습된 모델들은 훈련파라미터로 사용 x
kobart_model.eval(), mybart_model.eval()
kobart_model.to(device), mybart_model.to(device)

(BartForConditionalGeneration(
   (model): BartModel(
     (shared): BartScaledWordEmbedding(30000, 768, padding_idx=3)
     (encoder): BartEncoder(
       (embed_tokens): BartScaledWordEmbedding(30000, 768, padding_idx=3)
       (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
       (layers): ModuleList(
         (0-5): 6 x BartEncoderLayer(
           (self_attn): BartSdpaAttention(
             (k_proj): Linear(in_features=768, out_features=768, bias=True)
             (v_proj): Linear(in_features=768, out_features=768, bias=True)
             (q_proj): Linear(in_features=768, out_features=768, bias=True)
             (out_proj): Linear(in_features=768, out_features=768, bias=True)
           )
           (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
           (activation_fn): GELUActivation()
           (fc1): Linear(in_features=768, out_features=3072, bias=True)
           (fc2): Linear(in_features=3072, out_features=768, bias=True)
  

In [10]:
# 디코더 정의.
mydecoder = CrossAttentionDecoder(
    kobart_tokenizer_vocab_size=kobart_tokenizer.vocab_size,
    hidden_dim=768,
    num_layers=1,
    num_heads=1,
    dropout=0.1)

mydecoder.to(device)

CrossAttentionDecoder(
  (decoder): TransformerDecoder(
    (layers): ModuleList(
      (0): TransformerDecoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (multihead_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (linear1): Linear(in_features=768, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=768, bias=True)
        (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (norm3): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
        (dropout3): Dropout(p=0.1, inplace=False)
      )
    )
  )

In [11]:
# 예시용 출력임으로, detach해서 훈련에 영향주지 않도록한다.
batch = next(dataiter)
input_texts , output_texts = batch

sample_output_emb = get_korean_emb_vec(output_texts, kobart_tokenizer, kobart_model, device).detach()
sample_input_emb = get_encodedKr_emb_vec(input_texts, mybart_tokenizer, mybart_model, device).detach()


ground_truth_token_ids = kobart_tokenizer(output_texts, return_tensors="pt", padding="max_length",
                                          max_length=1026, truncation=True)["input_ids"].T.detach()


sample_output_emb.shape, sample_input_emb.shape, ground_truth_token_ids.shape # (torch.Size([1, 1026, 768]), torch.Size([1, 1026, 768]))

(torch.Size([2, 1026, 768]), torch.Size([2, 1026, 768]), torch.Size([1026, 2]))

In [12]:
# 의 결과 출력.
batch_size = 2
batch_max_len = 1026
for inputs, outputs in dataloader:
    # 🔥 배치 내 최대 `seq_len`을 찾기
    input_emb = get_encodedKr_emb_vec(input_texts=inputs, 
                                      mybart_tokenizer=mybart_tokenizer,
                                      mybart_model=mybart_model,
                                      device=device).permute(1, 0, 2)
    output_emb = get_korean_emb_vec(output_texts=outputs,
                                    kobart_tokenizer=kobart_tokenizer,
                                    kobart_model=kobart_model,
                                    device=device).permute(1, 0, 2)
    # 🔥 `tgt_mask`를 batch_max_len 기준으로 생성
    tgt_mask = nn.Transformer.generate_square_subsequent_mask(batch_max_len).to(device)

    print(input_emb.shape, output_emb.shape, tgt_mask.shape)
    output = mydecoder(input_emb, output_emb, tgt_mask=tgt_mask).detach()
    break

output.shape

torch.Size([1026, 2, 768]) torch.Size([1026, 2, 768]) torch.Size([1026, 1026])


torch.Size([1026, 2, 30000])

In [13]:
output_logits = output.reshape(-1, output.size(-1))

In [14]:
output_logits.shape

torch.Size([2052, 30000])

In [15]:
# 🔹 `output`에서 가장 높은 확률을 가진 토큰 인덱스 가져오기
predicted_token_ids = output_emb.argmax(dim=-1)  # (1026, 2)
print(predicted_token_ids.shape)
predicted_token_ids[1025]

torch.Size([1026, 2])


tensor([658, 658], device='cuda:0')

### 복원된 문장과 정답 비교하여 오차계산

In [16]:
# 🔹 정답 토큰 변환 (batch_size 단위로 `outputs`을 토큰화)
ground_truth_token_ids = kobart_tokenizer(outputs, return_tensors="pt", padding="max_length",
                                         max_length=batch_max_len, truncation=True)["input_ids"].T.to(device)
# 🔹 `.T`를 사용하여 `batch_size`를 두 번째 차원으로 맞춤 (max_len, batch_size)

# 🔹 예측값과 정답값 비교
correct = (predicted_token_ids == ground_truth_token_ids).sum().item()  # 맞춘 토큰 개수
total = batch_max_len * batch_size  # 전체 토큰 개수

# 🔹 정확도 계산
accuracy = correct / total
print(f"✅ 토큰 정확도: {accuracy:.4f} ({correct}/{total})")


✅ 토큰 정확도: 0.0000 (0/2052)


### 결과를 한국어로 변환

In [17]:
# 🔹 배치 단위로 복원
decoded_texts = []
batch_size = predicted_token_ids.shape[1]  # 배치 크기

for i in range(batch_size):  # 배치 크기만큼 반복
    token_ids = predicted_token_ids[:, i].tolist()  # 🔥 (max_len,) 형태로 변환
    decoded_text = kobart_tokenizer.decode(token_ids)  # 🔥 개별 문장 복원
    decoded_texts.append(decoded_text)

# 🔹 결과 확인
for i, text in enumerate(decoded_texts):
    print(f"🔹 복원된 문장 {i+1}: {text}")


🔹 복원된 문장 1: Pγγγγγγ<unused50>γP<unused50><unused50><unused50><unused50><unused50>PPPPPγPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPγγγγγγγγγγγγγγγγγγγγγγγγγγγγγγγγγγγγγ<unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50>PPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPγγγγγγγγγγγγγγγγγγγγγγγγγ<unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused50><unused5

In [18]:
# 🔹 배치 단위로 복원
decoded_texts = []
batch_size = ground_truth_token_ids.shape[1]  # 배치 크기

for i in range(batch_size):  # 배치 크기만큼 반복
    token_ids = ground_truth_token_ids[:, i].tolist()  # 🔥 (max_len,) 형태로 변환
    decoded_text = kobart_tokenizer.decode(token_ids) # 🔥 개별 문장 복원
    decoded_texts.append(decoded_text)

# 🔹 결과 확인
for i, text in enumerate(decoded_texts):
    print(f"🔹 실제문장 문장 {i+1}: {text}")


🔹 실제문장 문장 1: 매트리스와 베개에서 엄청난 곰팡이 냄새가 납니다. 또 다른 베개에서는 오래된 머리카락 냄새가 납니다. 침구 관리가 전혀 안 하는 듯합니다. 돈이 아깝습니다.<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [19]:
# 학습 설정
epochs = 1
batch_size = 2
batch_max_len = 1026
learning_rate = 5e-5

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.AdamW(mydecoder.parameters(), lr=learning_rate)

# Mixed Precision Training 설정
scaler = GradScaler()

# 학습 기록 저장용 리스트
loss_history = []
accuracy_history = []

# 학습 루프 시작
for epoch in range(epochs):
    mydecoder.train()
    total_loss = 0
    total_correct = 0
    total_tokens = 0

    tqdm_bar = tqdm(enumerate(dataloader), total=len(dataloader), desc=f"Epoch {epoch+1}/{epochs}")

    for step, (inputs, outputs) in tqdm_bar:
        optimizer.zero_grad()

        # 🔹 배치 내 최대 `seq_len` 찾기
        input_emb = get_encodedKr_emb_vec(inputs, mybart_tokenizer, mybart_model, device).permute(1, 0, 2)  # (torch.Size([2, 1026, 768])
        output_emb = get_korean_emb_vec(outputs, kobart_tokenizer, kobart_model, device).permute(1, 0, 2)   # (torch.Size([2, 1026, 768])
        tgt_mask = nn.Transformer.generate_square_subsequent_mask(batch_max_len).to(device)                 # torch.Size([1026, 1026])

        # 🔹 Mixed Precision Training 적용
        with autocast(device_type=device.type):
            decoder_output = mydecoder(input_emb, output_emb, tgt_mask=tgt_mask)  # decoder_output: (seq_len, batch_size, vocab_size)
            
            ground_truth_token_ids = kobart_tokenizer(outputs, return_tensors="pt", padding="max_length",
                                                    max_length=batch_max_len, truncation=True)["input_ids"].T.to(device)  # (seq_len, batch_size)
            

            # input1: (batch_size*max_len, vocab_size) 예측한 토큰 로짓값, input2: (batch_size*max_len) gt토큰 id번호
            loss = loss_fn(decoder_output.reshape(-1, decoder_output.size(-1)), ground_truth_token_ids.reshape(-1))

        
        # 🔹 Backpropagation
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # 🔹 손실 및 정답률 계산
        total_loss += loss.item()
        correct = (predicted_token_ids == ground_truth_token_ids).sum().item()  # 맞춘 토큰 개수
        total_correct += correct
        total_tokens += batch_max_len * batch_size

        accuracy = correct / (batch_max_len * batch_size)

        # 🔹 tqdm 업데이트
        tqdm_bar.set_postfix(loss=loss.item(), accuracy=accuracy)

        if step == 100:
            break
        # ----------------------------------------------------------------------------------------------------------------------------------------------------

    # 🔹 에포크당 평균 손실 및 정확도 기록
    epoch_loss = total_loss / len(dataloader)
    epoch_accuracy = total_correct / total_tokens

    loss_history.append(epoch_loss)
    accuracy_history.append(epoch_accuracy)

    print(f"\n✅ Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f} | Accuracy: {epoch_accuracy:.4f}")
    # ----------------------------------------------------------------------------------------------------------------------------------------------------

# 🔥 훈련 완료 후 손실 및 정답률 저장
torch.save(mydecoder, "trained_decoder_full.pth")

print("\n🎉 훈련 완료! 모델과 학습 기록이 저장되었습니다.")


Epoch 1/1:   1%|          | 100/16895 [00:50<2:21:13,  1.98it/s, accuracy=0, loss=0.0666]     



✅ Epoch 1/1 - Loss: 0.0079 | Accuracy: 0.0000

🎉 훈련 완료! 모델과 학습 기록이 저장되었습니다.


In [20]:
test_path = "datas/test.csv"
submission_path = "datas/submission.csv"
model_save_path = "trained_decoder.pth"


pretrained_decoder = torch.load("trained_decoder_full.pth")
pretrained_decoder.to(device)
pretrained_decoder.eval()


# ✅ 테스트 데이터 로드
df_test = pd.read_csv(test_path).sample(2)
test_input = df_test['input']

# ✅ 복원된 문장 저장 리스트
reconstructed_outputs = []

# ✅ Auto-regressive 방식에서 속도 최적화
with torch.no_grad():  # 그래디언트 계산 비활성화
    for input_text in tqdm(test_input, desc="복구 진행 중"):
        # 🔹 난독화된 문장 임베딩 벡터 생성
        input_emb = get_encodedKr_emb_vec([input_text], mybart_tokenizer, mybart_model, device).permute(1, 0, 2)

        # 🔹 디코딩할 첫 번째 입력 토큰 설정 (시작 토큰 `<sos>`)
        start_token_id = kobart_tokenizer.bos_token_id
        end_token_id = kobart_tokenizer.eos_token_id
        current_token_ids = torch.tensor([[start_token_id]], dtype=torch.long).to(device)

        # 🔹 Greedy Decoding을 위한 토큰 생성
        generated_tokens = []

        # 🔹 최대 `batch_max_len` 길이만큼 반복
        for _ in range(batch_max_len):
            # 🔹 현재까지의 생성된 토큰을 `kobart_model`로 인코딩
            output_emb = kobart_model(current_token_ids).encoder_last_hidden_state.permute(1, 0, 2)

            # 🔹 `tgt_mask` 생성 (자동회귀 방식 적용)
            tgt_mask = nn.Transformer.generate_square_subsequent_mask(output_emb.size(0)).to(device)

            # 🔹 디코더 예측 수행
            decoder_output = pretrained_decoder(input_emb, output_emb, tgt_mask=tgt_mask)  # (seq_len, batch_size, vocab_size)

            # 🔹 가장 확률 높은 토큰 선택
            next_token_id = decoder_output.argmax(dim=-1)[-1, 0].item()  # 마지막 토큰 선택

            # 🔹 생성된 토큰 저장
            generated_tokens.append(next_token_id)

            # 🔹 종료 조건: `<eos>` 토큰이 생성되면 중단
            if next_token_id == end_token_id:
                break

            # 🔹 다음 입력으로 사용 (효율적으로 변환)
            current_token_ids = torch.cat([current_token_ids, torch.tensor([[next_token_id]], dtype=torch.long).to(device)], dim=1)

        # 🔹 토큰 ID를 한국어 문장으로 변환
        reconstructed_text = kobart_tokenizer.decode(generated_tokens)

        # 🔹 디버깅 정보 출력 (1개만 확인)
        if len(reconstructed_outputs) < 1:
            print(f"\n🛠️ [디버깅] 원본: {input_text}")
            print(f"🛠️ [디버깅] 예측된 토큰 ID: {generated_tokens}")
            print(f"🛠️ [디버깅] 복원된 문장: {reconstructed_text}")

        reconstructed_outputs.append(reconstructed_text)

# ✅ `submission.csv` 생성
df_submission = pd.DataFrame({"ID": df_test.index.map(lambda x: f"TEST_{x:04d}"), "output": reconstructed_outputs})
df_submission.to_csv(submission_path, index=False)

print("\n🎉 `submission.csv` 파일이 생성되었습니다!")

/tmp/ipykernel_150021/2090858286.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_decoder = torch.load("trained_decoder_full.pth")
복구 진행 중:  50%|█████     | 1/


🛠️ [디버깅] 원본: 껙쉴엣써 본윈눈 부는 청먈 쬐꼰. 홅뗄림 옮레퇴댜 폰닉꺄 젼췌척읕료 뇨휴확돼교 싶섦일 옜껑씨란는 첨운 앝쉽징많, 콴뤽눈 맵욹 청걺학꼐 짧 툇얽 잇타. 쥑건툴읫 읗뎅냐 셔빛슨눈 뮤쩍 췬쩔햐댜. 뭄었뽀닫토 맘뭬 들엇뎐 꼇순 룸섶빚수 뭬뉼. 행묾람먼 갛출. 쥑굼카쥐 머컸턴 랑면 츙예 위툐록 곧굽찐 람먼눈 쩌욺 멱억뵨 듣. 젼폭, 초껙관찬, 쌔유, 꽃께 둥윌 씬션한 헤샨묾위 쩡먈 멱귀 좋궤 랴먼과 얼웁려쪄 냘욘는뗑, 륨엘석 앝씩꿀료 쉭껴 멱깁 탁윅댜. 삐잘, 찍퀸, 깖뛰됴 얗이 뿌쥠학꼰 맡됴 좋댜. 롭삣쯩의 옳큄술랴는 야이륏쉿 폅엡섶 블런췰롤 군냘의 쒜뿌 츄천 쎄뚝 뭬눗량 팬께윗쿠를 식커 먹였눈테, 뚜 걔 댜 캉쮸! 수톄익쿤는 육질곽 꾸웠넨는 청토됴 딱 좋앗교, 뺀꼐잊쿠는 난쌩쪄움 폰눈 삐주열료 풍췸갤철렴 냐욘는 엮누 폿똥의 펜켕익끙와눈 짜얹인 닮른닿. 먀찔 몽씰한 뭏객쿠름쳐렴 한껐 풉풂령써 낮온눈뎀 잎벴 넣우면 쑨쉭칸웨 샬쌀 뇩눈댜. 닳음옘 뿌삵눼 냇령요면 블런찌는 억귀셔 콕 먹겨악겟타. 뮈린 챵갸 촉 옛약운 퓔쑤. 객쒸릴 탔쇼 촙쿄 씨셜뤼 놉훌퇸 뿌뿐닢 잇언 별 4께 추러타 호텔 윅칩, 젼먕, 셥피쓱 북뿌녜셔 념뭏 많쪽순렵귀웨 별 5깨룰 앉 줆 쑥캬 엎따. ㅎㅎ
🛠️ [디버깅] 예측된 토큰 ID: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3

복구 진행 중: 100%|██████████| 2/2 [01:49<00:00, 54.92s/it]


🎉 `submission.csv` 파일이 생성되었습니다!


In [21]:
# input1(난독화 텍스트) 임베딩 벡터 생성용 함수 (배치 단위 지원)
def get_encodedKr_emb_vec(input_texts, mybart_tokenizer, mybart_model, device, max_length=1026):
    '''
    mybart_model는 eval() 모드여야 한다.
    input_texts: List of input strings (배치 단위)
    '''
    input_ids_batch = []
    attention_mask_batch = []

    for input_text in input_texts:
        input_encoded = mybart_tokenizer.encode(input_text)
        input_ids = input_encoded.ids[:max_length]
        attention_mask = [1] * len(input_ids)

        pad_id = mybart_tokenizer.token_to_id("<pad>")
        input_ids += [pad_id] * (max_length - len(input_ids))
        attention_mask += [0] * (max_length - len(attention_mask))

        input_ids_batch.append(input_ids)
        attention_mask_batch.append(attention_mask)

    # (batch_size, seq_len) 형태로 변환
    input_ids = torch.tensor(input_ids_batch, dtype=torch.long).to(device)
    attention_mask = torch.tensor(attention_mask_batch, dtype=torch.long).to(device)

    input_emb = mybart_model.encoder(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
    return input_emb  # (batch_size, seq_len, hidden_dim)